### Chroma
* 오픈소스 벡터 데이터베이스
* 개발자의 생산성과 행복에 초점을 맞춤

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일 로드
loader1 = TextLoader("./data/nlp-keywords.txt")
loader2 = TextLoader("./data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

print(len(split_doc1))
print(len(split_doc2))

11
6


### VectorStore 생성

**from_documents()**
* 문서 리스트로부터 벡터 저장소를 생성한다.

**매개변수**
* documents : 벡터 저장소에 추가할 문서 리스트(필수)
* collection_name : 생성할 컬렉션 이름(필수)
* embeddings : 임베딩 함수
* ids : 문서 id 리스트
* persist_directory : 컬렉션을 저장할 디렉토리
* client_settings : Chroma 사용자 설정
* clinet : 사용자 인스턴스
* collection_metadata : 컬렉션 구성 정보

In [7]:
db = Chroma.from_documents(
    documents=split_doc1,
    embedding=OpenAIEmbeddings(),
    collection_name="my_db"
)

In [9]:
# 저장 경로
DB_PATH = "./chroma_db"

# 파일로 저장
persist_db = Chroma.from_documents(
    documents=split_doc1,
    embedding=OpenAIEmbeddings(),
    collection_name="my_db",
    persist_directory=DB_PATH
)

In [12]:
persist_db.get()

{'ids': ['dcb8c37f-199a-42a8-833d-ae63c2faa0aa',
  '6d7a5703-1b4a-47c8-9667-b05e3ec9ae48',
  '22952653-e0e1-4ac9-bf9d-309553412848',
  'd7f8e0f1-79dd-47c9-8343-9521d29177b3',
  '25cbb7e5-4636-452b-a13d-49a3ec24caf6',
  'd0708441-01d6-4b00-a47c-22493ad5c359',
  '5da69ed7-dc0a-4bd5-adb6-38ea64834b9e',
  'a5ec653f-df06-47ca-8dbc-f81260ef11f8',
  'dd11f931-df89-4f4f-8e6f-6bf11d600f57',
  'e6000b03-6af4-4fec-80c4-aee044864b04',
  'ebec9b0e-bb74-4ae3-b4eb-d1a8c004b67c'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 

**from texts**
* 텍스트 리스트로 벡터 저장소를 생성

**매개변수**
*  texts : 벡터 저장소에 추가할 텍스트 리스트(필수)
* collection_name : 생성할 컬렉션 이름(기본값 : _LANGCHAIN_COLLECTION_NAME)
* embeddings : 임베딩 함수
* ids : 문서 id 리스트
* persist_directory : 컬렉션을 저장할 디렉토리
* client_settings : Chroma 사용자 설정
* clinet : 사용자 인스턴스
* collection_metadata : 컬렉션 구성 정보

In [25]:
db2 = Chroma.from_texts(
    ["안녕하세요. 반갑습니다. 오늘 메뉴는 뭔가요?", "제 이름은 성민입니다."],
    embedding=OpenAIEmbeddings()
)

In [26]:
db2.get()

{'ids': ['7e89344b-5d0d-4217-b72a-07035b3cdcdc',
  '45dde081-a364-49d2-8f05-e61e22156dbb'],
 'embeddings': None,
 'documents': ['안녕하세요. 반갑습니다. 오늘 메뉴는 뭔가요?', '제 이름은 성민입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**similarity_search**
* Chroma 데이터베이스에서 유사도 검색을 수행한다.
* 주어진 쿼리와 가장 유사한 문서를 반환한다.
* 매개변수
    * query : 검색할 쿼리 텍스트
    * k : 반환할 결과의 수(기본값은 4)
    * filter : 메타데이터로 필터링(기본값은 None)
* 참고
    * 리턴 시 점수정보없이 문서만 반환된다.
    * 점수정보를 얻고 싶으면 similarity_search_with_score()

In [17]:
db.similarity_search("csv에 대해 알려줘.", k=2)

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n연관키워드: 데이터 형식, 파일 처리, 데이터 교환\n\nJSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nHuggingFace'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore\n\n정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 

**add_documents()**
* 벡터 스토어에 문서 추가를 할 수 있다.
* 매개변수
    * documents : 벡터 저장소에 추가할 문서 리스트
    * **kwargs : 추가 키워드 인자
        * ids : 문서 ID 리스트

In [18]:
from langchain_core.documents import Document

db.add_documents(
    [
        Document(
            page_content="안녕하세요. 이번에는 문서를 추가해 볼 거에요.",
            metadata={"source": "mydata.txt"},
            id="1"
        )
    ]
)

['1']

In [19]:
db.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['안녕하세요. 이번에는 문서를 추가해 볼 거에요.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**add_texts**
* 텍스트를 임베딩하고 벡터 저장소에 추가
* 매개변수
    *  texts : 벡터 저장소에 추가할 텍스트 리스트
    * metadatas : 메타데이터 리스트(기본값은 None)
    * ids : 문서 ID 리스트(기본값 None)
        * 입력하지 않으면 UUID를 사용해 자동으로 생성된다.
        * 기존의 id에 추가하면 덮어 씌워지게 된다.

In [20]:
db.add_texts(
    ["이전에 추가한 문서를 덮어 씌우기", "기존 문서는 삭제됩니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["1", "2"]
)

['1', '2']

In [23]:
print(db.get(["1"]))
print(db.get(["2"]))

{'ids': ['1'], 'embeddings': None, 'documents': ['이전에 추가한 문서를 덮어 씌우기'], 'uris': None, 'data': None, 'metadatas': [{'source': 'mydata.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
{'ids': ['2'], 'embeddings': None, 'documents': ['기존 문서는 삭제됩니다.'], 'uris': None, 'data': None, 'metadatas': [{'source': 'mydata.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
